In [32]:
# python libraries
import numpy as np
from IPython.display import HTML, display
# pydrake imports
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         Linearize, LinearQuadraticRegulator, LogVectorOutput,
                         MeshcatVisualizer, ModelVisualizer, Parser, Simulator, StartMeshcat)

from underactuated import running_as_notebook

In [33]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://791ea483-dc7d-4417-8d6a-d18ad7d578e9.deepnoteproject.com/7001/


## Trajectory Optimization of a Free-Falling Cat

In [34]:
# segLength = 0.175m
# segDiam = 0.15m
# segMass = 1kg
# segXverseMOI = 1kg-m^2
# JIratio = 0.25 (xverse to axial MOI ratio)
# tau_max = 40Nm (max torque)
# I_xx = segAxialMOI = JIratio*segXverseMOI
# I_yy = I_zz = segXverseMOI
# I_xy = I_xz = I_yz = 0

# dummy sphere no mass for gamma (hunch)
S1 = """
  <link name="S1">

    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <mass value="0" />
    </inertial>

    <visual>
      <origin xyz="0 0 0"/>
      <geometry>
        <sphere radius=".01"/>
      </geometry>
      <material>
        <color rgba="0 1 1 1" />
      </material>
    </visual>
  </link>
"""

# dummy sphere no mass for psi/phi (twist)
S2 = """
  <link name="S2">

    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <mass value="0" />
    </inertial>

    <visual>
      <origin xyz="0 0 0"/>
      <geometry>
        <sphere radius=".01"/>
      </geometry>
      <material>
        <color rgba="0 1 1 1" />
      </material>
    </visual>
  </link>
"""

B1 = """
  <link name="B1">

    <inertial>
      <origin xyz="0.0875 0 0" rpy="0 0 0"/>
      <mass value="1" />
      <inertia ixx="0.25"  ixy="0"  ixz="0" iyy="1" iyz="0" izz="1" />
    </inertial>

    <visual>
      <origin xyz="0.0875 0 0" rpy="1.57 0 1.57"/>
      <geometry>
        <cylinder radius=".075" length="0.175"/>
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0.0866 -0.05 -0.14" rpy="0 0 -0.5235"/>
      <geometry>
        <box size=".1 .02 .02"/>
      </geometry>
      <material>
        <color rgba="1 0 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0.0866 0.05 -0.14" rpy="0 0 0.5235"/>
      <geometry>
        <box size=".1 .02 .02"/>
      </geometry>
      <material>
        <color rgba="1 0 0 1" />
      </material>
    </visual>


  </link>
"""

B2 = """
  <link name="B2">

    <inertial>
      <origin xyz="-0.0875 0 0" rpy="0 0 3.14159"/>
      <mass value="1" />
      <inertia ixx="0.25"  ixy="0"  ixz="0" iyy="1" iyz="0" izz="1" />
    </inertial>

    <visual>
      <origin xyz="-0.0875 0 0" rpy="1.57 0 -1.57"/>
      <geometry>
        <cylinder radius=".075" length="0.175"/>
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0.0866 -0.05 0.14" rpy="0 0 -0.5235"/>
      <geometry>
        <box size=".1 .02 .02"/>
      </geometry>
      <material>
        <color rgba="1 0 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0.0866 0.05 0.14" rpy="0 0 0.5235"/>
      <geometry>
        <box size=".1 .02 .02"/>
      </geometry>
      <material>
        <color rgba="1 0 0 1" />
      </material>
    </visual>
  </link>
"""

# old messed up model
# B1 = """
#   <link name="B1">

#     <inertial>
#       <origin xyz="-0.0875 0 0" rpy="0 1.57 0"/>
#       <mass value="1" />
#       <inertia ixx="0.25"  ixy="0"  ixz="0" iyy="1" iyz="0" izz="1" />
#     </inertial>

#     <visual>
#       <origin xyz="0 0 -0.0875"/>
#       <geometry>
#         <cylinder radius=".075" length="0.175"/>
#       </geometry>
#       <material>
#         <color rgba="0 1 0 1" />
#       </material>
#     </visual>

#     <visual>
#       <origin xyz="0.0866 -0.05 -0.14" rpy="0 0 -0.5235"/>
#       <geometry>
#         <box size=".1 .02 .02"/>
#       </geometry>
#       <material>
#         <color rgba="1 0 0 1" />
#       </material>
#     </visual>

#     <visual>
#       <origin xyz="0.0866 0.05 -0.14" rpy="0 0 0.5235"/>
#       <geometry>
#         <box size=".1 .02 .02"/>
#       </geometry>
#       <material>
#         <color rgba="1 0 0 1" />
#       </material>
#     </visual>


#   </link>
# """



    # <visual>
    #   <origin xyz = "0.08 0 -0.14" rpy="0 0 -0.785">
    #   <geometry>
    #     <box size="0.08 0.02 0.02" />
    #   </geometry>
    #   <material>
    #     <color rgba="0 1 1 1" />
    #   </material>
    # </visual>

    # <visual>
    #   <origin xyz = "0 0 -0.14" rpy="0 0 0.785">
    #   <geometry>
    #     <box size="0.12 0.02 0.02" />
    #   </geometry>
    #   <material>
    #     <color rgba="0 1 1 1" />
    #   </material>
    # </visual>


# B2 = """
#   <link name="B2">

#     <inertial>
#       <origin xyz="0 0 0.0875" rpy="0 -1.57 0"/>
#       <mass value="1" />
#       <inertia ixx="0.25"  ixy="0"  ixz="0" iyy="1" iyz="0" izz="1" />
#     </inertial>

#     <visual>
#       <origin xyz="0 0 0.0875" rpy="0 -3.14159 0"/>
#       <geometry>
#         <cylinder radius=".075" length="0.175"/>
#       </geometry>
#       <material>
#         <color rgba="0 1 0 1" />
#       </material>
#     </visual>

#     <visual>
#       <origin xyz="0.0866 -0.05 0.14" rpy="0 0 -0.5235"/>
#       <geometry>
#         <box size=".1 .02 .02"/>
#       </geometry>
#       <material>
#         <color rgba="1 0 0 1" />
#       </material>
#     </visual>

#     <visual>
#       <origin xyz="0.0866 0.05 0.14" rpy="0 0 0.5235"/>
#       <geometry>
#         <box size=".1 .02 .02"/>
#       </geometry>
#       <material>
#         <color rgba="1 0 0 1" />
#       </material>
#     </visual>
#   </link>
# """

# maybe add legs or smth

In [35]:
# S_joint = """
#   <joint name="Sjoint" type="planar">
#     <parent link="world" />
#     <child link="S" />
#     <origin rpy="0 -1.57 0" xyz="0 0 0" />
#   </joint>
# """
S1_joint = """
  <joint name="S1joint" type="prismatic">
    <parent link="world" />
    <child link="S1" />
    <origin rpy="0 0 0" xyz="0 0 0" />
    <axis xyz="0 0 1" />
  </joint>
"""

S2_joint = """
  <joint name="S2joint" type="fixed">
    <parent link="S1" />
    <child link="S2" />
    <origin rpy="0 0 0" xyz="0 0 0" />
  </joint>
"""

In [36]:
spine_joint1 = """
  <joint name="gamma1" type="revolute">
    <parent link="S1" />
    <child link="B1" />
    <axis xyz="0 -1 0" />
    <origin xyz="0 0 0"/>
    <limit lower="-1.57" upper="1.57"/>
  </joint>
"""

spine_joint2 = """
  <joint name="gamma2" type="revolute">
    <parent link="S1" />
    <child link="B2" />
    <axis xyz="0 1 0" />
    <origin xyz="0 0 0"/>
    <limit lower="-1.57" upper="1.57"/>
  </joint>
"""

In [40]:
revolve_joint1 = """
  <joint name="psi1" type="continuous">
    <parent link="B1" />
    <child link="S2" />
    <axis xyz="1 0 0" />
    <origin xyz="0 0 0"/>
  </joint>
"""

revolve_joint2 = """
  <joint name="psi2" type="continuous">
    <parent link="B2" />
    <child link="S2" />
    <axis xyz="1 0 0" />
    <origin xyz="0 0 0"/>
  </joint>
"""

### Transmissions
Lastly, we come to the transmission component. Here the only controlled input is a force applied in the x direction on the cart base. 

In [41]:
# # DO NOT MODIFY
hunch1_transmission = """
  <transmission type="SimpleTransmission" name="hunch1_torque">
    <actuator name="h1_torque" />
    <joint name="gamma1" />
  </transmission>
"""

hunch2_transmission = """
  <transmission type="SimpleTransmission" name="hunch2_torque">
    <actuator name="h2_torque" />
    <joint name="gamma2" />
  </transmission>
"""

## Assembling the URDF

Now we have all the components necessary to construct a URDF for the cart-pole. Below is a function that assembles these components and adds a header and an ender to wrap them into one system. Take a few minutes to see how the generated urdf looks like.

In [42]:
cat_urdf = f"""
<?xml version="1.0"?><robot name="MyCat">
{S1}
{S2}
{B1}
{B2}
{S1_joint}
{S2_joint}
{spine_joint1}
{spine_joint2}
{revolve_joint1}
{revolve_joint2}
  </robot>
</xml>
"""
# {S_joint}
print(cat_urdf)

# {hunch1_transmission}
# {hunch2_transmission}


<?xml version="1.0"?><robot name="MyCat">

  <link name="S1">

    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <mass value="0" />
    </inertial>

    <visual>
      <origin xyz="0 0 0"/>
      <geometry>
        <sphere radius=".01"/>
      </geometry>
      <material>
        <color rgba="0 1 1 1" />
      </material>
    </visual>
  </link>


  <link name="S2">

    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <mass value="0" />
    </inertial>

    <visual>
      <origin xyz="0 0 0"/>
      <geometry>
        <sphere radius=".01"/>
      </geometry>
      <material>
        <color rgba="0 1 1 1" />
      </material>
    </visual>
  </link>


  <link name="B1">

    <inertial>
      <origin xyz="0.0875 0 0" rpy="0 0 0"/>
      <mass value="1" />
      <inertia ixx="0.25"  ixy="0"  ixz="0" iyy="1" iyz="0" izz="1" />
    </inertial>

    <visual>
      <origin xyz="0.0875 0 0" rpy="1.57 0 1.57"/>
      <geometry>
        <cylinder radius=".075" length="0.175"

In [43]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(cat_urdf, 'urdf')
visualizer.Run(loop_once=not running_as_notebook)
meshcat.Delete()
meshcat.DeleteAddedControls()

RuntimeError: When creating a model, an attempt was made to add two inboard joints to the same body; this is not allowed. One possible cause might be attempting to weld a robot to World somewhere other than its base link; see Drake issue #17429 for discussion and work-arounds, e.g., reversing some joint parent/child directions. Another possible cause might be attempting to form a kinematic loop using joints; to create a loop, consider using a LinearBushingRollPitchYaw instead of a joint.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=791ea483-dc7d-4417-8d6a-d18ad7d578e9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>